Júlia Dalmau,
Ariadna Gonzalez i 
Mireia Cuenca

In [94]:
#Imports

from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
import time
import string
import nltk
import json

In [20]:
#Load data into memory

docs_path = 'C:/Users/mirei/Downloads/IRWA - Project-20221011/data/data/tw_hurricane_data.json' #path of tw_hurricane_data.json
with open(docs_path) as fp:
    lines = fp.readlines()
    
#Print lines[0] to see the structure of the data 

print(lines[0])

{"created_at": "Fri Sep 30 18:39:08 +0000 2022", "id": 1575918182698979328, "id_str": "1575918182698979328", "full_text": "So this will keep spinning over us until 7 pm\u2026go away already. #HurricaneIan https://t.co/VROTxNS9rz", "truncated": false, "display_text_range": [0, 76], "entities": {"hashtags": [{"text": "HurricaneIan", "indices": [63, 76]}], "symbols": [], "user_mentions": [], "urls": [], "media": [{"id": 1575918178261254162, "id_str": "1575918178261254162", "indices": [77, 100], "media_url": "http://pbs.twimg.com/media/Fd7JO8pXwBI9HPw.jpg", "media_url_https": "https://pbs.twimg.com/media/Fd7JO8pXwBI9HPw.jpg", "url": "https://t.co/VROTxNS9rz", "display_url": "pic.twitter.com/VROTxNS9rz", "expanded_url": "https://twitter.com/suzjdean/status/1575918182698979328/photo/1", "type": "photo", "sizes": {"small": {"w": 521, "h": 680, "resize": "fit"}, "thumb": {"w": 150, "h": 150, "resize": "crop"}, "medium": {"w": 919, "h": 1200, "resize": "fit"}, "large": {"w": 1284, "h": 1677, "r

In [95]:
#Transform data into a dict to make easier the code

datos_diccionario = [json.loads(line) for line in lines]

#Now if we want to access to the screen_name of a user we colud do it like this

datos_diccionario[0]['user']['screen_name']

'suzjdean'

In [96]:
def build_terms(line):

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))

    #Transform in lowercase
    line=  line.lower()
    
    #Removing punctuation marks
    line= line.translate(str.maketrans('', '', string.punctuation)) 
    
    #Tokenize the text to get a list of terms
    line=  line.split(" ")
    
    #Removing the stopwords
    line=[x for x in line if x not in stop_words]
    
    #Perform stemming
    line=[stemmer.stem(x) for x in line]

    return line

In [122]:
def create_index(lines):
    
    index = defaultdict(list)
    title_index = {}
    
    for line in lines:
    
        terms = build_terms(line['full_text'])
        
        page_id = line['id']
        title = line['id_str']
        title_index[page_id]=title
        
        # The final output must return Tweet, Username, Date, Hashtags, Likes, Retweets and Url
        args = [line["id_str"], line['user']['screen_name'], line['created_at'], line['entities']['hashtags'], line['user']['favourites_count'], line['retweet_count'], line['entities']['urls']]
        
        for term in terms:
            index[term].append(args)
                    
    return index, title_index

In [123]:
#Create index

index, title_index = create_index(datos_diccionario)

In [145]:
def search(query, index):
   
    query = build_terms(query)
    docs = []
    for term in query:
        try:
            #Term is in the index
            keys = [i for i in index.keys()]
            term_docs = [index[t] for t in keys if t==term]
            docs=term_docs[0]
            
        except:
            #Term is not in index
            pass
    return docs

In [148]:
print("Insert your query (i.e.: Computer Science):\n")
query = input()
docs = search(query, index)
top = 10


print("\n======================\nSample of {} results out of {} for the searched query:\n".format(top, len(docs)))
for d_id in docs[:top]:
    print('Tweet_id: ', d_id[0], ' Username: ', d_id[1], ' Date: ', d_id[2], ' Hashtags: ', d_id[3], ' Likes: ', d_id[4], ' Url: ', d_id[5])

Insert your query (i.e.: Computer Science):

pier

Sample of 10 results out of 36 for the searched query:

Tweet_id:  1575918018235813888  Username:  PalmettoMae  Date:  Fri Sep 30 18:38:29 +0000 2022  Hashtags:  [{'text': 'HurricaneIan', 'indices': [262, 275]}]  Likes:  108899  Url:  0
Tweet_id:  1575918018235813888  Username:  PalmettoMae  Date:  Fri Sep 30 18:38:29 +0000 2022  Hashtags:  [{'text': 'HurricaneIan', 'indices': [262, 275]}]  Likes:  108899  Url:  0
Tweet_id:  1575918018235813888  Username:  PalmettoMae  Date:  Fri Sep 30 18:38:29 +0000 2022  Hashtags:  [{'text': 'HurricaneIan', 'indices': [262, 275]}]  Likes:  108899  Url:  0
Tweet_id:  1575918018235813888  Username:  PalmettoMae  Date:  Fri Sep 30 18:38:29 +0000 2022  Hashtags:  [{'text': 'HurricaneIan', 'indices': [262, 275]}]  Likes:  108899  Url:  0
Tweet_id:  1575917717600681984  Username:  jf_darwin  Date:  Fri Sep 30 18:37:17 +0000 2022  Hashtags:  [{'text': 'HurricaneIan', 'indices': [0, 13]}, {'text': 'wxtwitte